# Retrieve climate data through the meteo.cat API

In [21]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 


In [56]:
# API key
key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
url = 'https://api.meteo.cat/xema/v1' + '/estacions/X8/variables/mesurades/metadades'

headers = {'Accept': 'application/json', 'X-API-KEY': key}

r = requests.get(url)

display(r.status_code)
display(r.json())

403

In [57]:
url = 'https://api.meteo.cat/quotes/v1/consum-actual'


headers = {'X-API-KEY': 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'}
requests.get(url, headers=headers).json()

{'client': {'nom': 'Pau Torrente Badia UB'},
 'plans': [{'nom': 'XEMA_750 OD',
   'periode': 'Mensual',
   'maxConsultes': 750,
   'consultesRestants': 0,
   'consultesRealitzades': 750},
  {'nom': 'Predicció_100',
   'periode': 'Mensual',
   'maxConsultes': 100,
   'consultesRestants': 100,
   'consultesRealitzades': 0},
  {'nom': 'XDDE_250',
   'periode': 'Mensual',
   'maxConsultes': 250,
   'consultesRestants': 250,
   'consultesRealitzades': 0},
  {'nom': 'Quota',
   'periode': 'Mensual',
   'maxConsultes': 300,
   'consultesRestants': 300,
   'consultesRealitzades': 0},
  {'nom': 'Referència Bàsic',
   'periode': 'Mensual',
   'maxConsultes': 2000,
   'consultesRestants': 2000,
   'consultesRealitzades': 0}]}

In [49]:
key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

headers = {'Accept': 'application/json', 'X-API-KEY': key}

r = requests.get(url, headers=headers)

data = r.json()[0]

variables = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]

date_rain = [{'timestamp':int(dt.datetime.strptime(d['data'], '%Y-%m-%dT%H:%MZ').timestamp()), 'mm_precip':d['valor']} for d in variables ]

date_rain

KeyError: 0

In [43]:
import time

start_date = dt.date(2019, 1, 1)
end_date = dt.date(2019, 1, 3)

start_date < end_date

delta = dt.timedelta(days=1)

freq = 1/20

rain = []

day = start_date

while day <= end_date:

    day_string = day.strftime('%Y/%m/%d')

    url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/' + day_string 

    print(day_string)

    headers = {'Accept': 'application/json', 'X-API-KEY': key}

    data = requests.get(url, headers=headers).json()[0]

    variables = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]

    date_rain = [{'timestamp':int(dt.datetime.strptime(d['data'], '%Y-%m-%dT%H:%MZ').timestamp()), 'mm_precip':d['valor']} for d in variables ]

    rain.extend(date_rain)

    day += delta

    time.sleep(freq)



2019/01/01


KeyError: 0

# Station localization on a map using the folium library

In [18]:
import pandas as pd
import datetime as dt

gener_info = pd.read_csv('data/2021_01_Gener_BicingNou_INFORMACIO.csv')

gener_info.head()

gener_info.shape

(4505063, 13)

In [17]:
gener_estacions = pd.read_csv('data/2021_01_Gener_BicingNou_ESTACIONS.csv')

gener_estacions.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,last_updated,ttl
0,1,43,43,0,3,2020-12-31 23:56:46,True,IN_SERVICE,1,1,1,1609455626,22
1,2,17,15,2,10,2020-12-31 23:57:51,True,IN_SERVICE,1,1,1,1609455626,22
2,3,3,3,0,22,2020-12-31 23:56:48,True,IN_SERVICE,1,1,1,1609455626,22
3,4,15,14,1,6,2020-12-31 23:57:38,True,IN_SERVICE,1,1,1,1609455626,22
4,5,8,8,0,31,2020-12-31 23:57:21,True,IN_SERVICE,1,1,1,1609455626,22


In [7]:
locations = gener_info[['name', 'lat', 'lon']].groupby('name').mean().reset_index()

locations.loc[1]['lat']

locations.shape

(506, 3)

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import folium

from folium.plugins import HeatMap

map = folium.Map(location=[41.3870154, 2.1700471], zoom_start=13)

for i in range(locations.shape[0]):
    folium.Marker([locations.loc[i]['lat'], locations.loc[i]['lon']], popup=locations.loc[i]['name']).add_to(map)

map


